# Tratamento Inicial do Endereço Livre - NER

Este notebook implementa o reconhecimento de entidades nomeadas (NER) para estruturar endereços livres em componentes: LOGRADOURO, NÚMERO, BAIRRO, COMPLEMENTO.

In [ ]:
# Importar configurações do notebook anterior
%run ./00_configuracao_inicial.ipynb

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover
import re

# Schema para endereços livres de entrada
schema_endereco_livre = StructType([
    StructField("id", StringType(), True),
    StructField("endereco_livre", StringType(), True),
    StructField("uf", StringType(), True),
    StructField("cidade", StringType(), True),
    StructField("timestamp", TimestampType(), True)
])

In [ ]:
# UDF para extrair número do endereço
def extrair_numero(texto):
    """Extrai número do endereço usando regex"""
    if not texto:
        return None
    
    # Padrões comuns de números em endereços
    padroes = [
        r'\b(\d+[A-Za-z]?)\b',  # Número simples ou com letra (ex: 120, 120A)
        r'n[úu]mero\s+(\d+)',
        r'n[úu]m\.\s+(\d+)',
        r'n[º°]\s*(\d+)',
    ]
    
    for padrao in padroes:
        match = re.search(padrao, texto, re.IGNORECASE)
        if match:
            return match.group(1) if len(match.groups()) > 0 else match.group(0)
    
    return None

udf_extrair_numero = udf(extrair_numero, StringType())

In [ ]:
# UDF para identificar tipo de logradouro
def identificar_tipo_logradouro(texto):
    """Identifica tipo de logradouro (RUA, AVENIDA, etc)"""
    if not texto:
        return None
    
    tipos = {
        'RUA': ['rua', 'r\.', 'r '],
        'AVENIDA': ['avenida', 'av\.', 'av ', 'ave\.'],
        'ESTRADA': ['estrada', 'est\.', 'est '],
        'RODOVIA': ['rodovia', 'rod\.', 'br-'],
        'PRAÇA': ['praça', 'pra\s*ca', 'pc\.'],
        'TRAVESSA': ['travessa', 'trav\.', 'tv\.'],
        'ALAMEDA': ['alameda', 'al\.', 'alm\.'],
        'VIELA': ['viela', 'vl\.'],
        'BECO': ['beco', 'bc\.'],
        'LARGO': ['largo', 'lg\.'],
        'PARQUE': ['parque', 'pq\.'],
    }
    
    texto_lower = texto.lower()
    
    for tipo, padroes in tipos.items():
        for padrao in padroes:
            if re.search(r'\b' + padrao + r'\b', texto_lower):
                return tipo
    
    return 'LOGRADOURO'  # Tipo genérico

udf_identificar_tipo = udf(identificar_tipo_logradouro, StringType())

In [ ]:
# UDF para extrair nome do logradouro (remove tipo e número)
def extrair_nome_logradouro(texto, tipo_logradouro):
    """Extrai o nome do logradouro removendo tipo e número"""
    if not texto:
        return None
    
    # Remove tipo de logradouro
    texto_limpo = texto
    if tipo_logradouro:
        tipos_remover = ['rua', 'r\.', 'r ', 'avenida', 'av\.', 'av ', 'estrada', 'est\.']
        for tipo in tipos_remover:
            texto_limpo = re.sub(r'\b' + tipo + r'\b', '', texto_limpo, flags=re.IGNORECASE)
    
    # Remove número
    texto_limpo = re.sub(r'\b\d+[A-Za-z]?\b', '', texto_limpo)
    texto_limpo = re.sub(r'n[úu]mero\s+\d+', '', texto_limpo, flags=re.IGNORECASE)
    
    # Remove palavras comuns
    texto_limpo = re.sub(r'\b(perto|próximo|próx|de|da|do|dos|das|na|no|em)\b', '', texto_limpo, flags=re.IGNORECASE)
    
    return texto_limpo.strip()

udf_extrair_nome_log = udf(extrair_nome_logradouro, StringType())

In [ ]:
# UDF para extrair bairro (geralmente após vírgula ou palavras-chave)
def extrair_bairro(texto):
    """Extrai bairro do endereço livre"""
    if not texto:
        return None
    
    # Padrões comuns para bairro
    padroes_bairro = [
        r',\s*([^,]+?)(?:,|$)',  # Após primeira vírgula
        r'\b(bairro|b\.|distrito)\s+([^,]+?)(?:,|$)',  # Palavra-chave bairro
        r'\b(centro|zona\s+(norte|sul|leste|oeste))\b',  # Bairros comuns
    ]
    
    for padrao in padroes_bairro:
        match = re.search(padrao, texto, re.IGNORECASE)
        if match:
            bairro = match.group(1) if len(match.groups()) >= 1 else match.group(0)
            # Remove palavras comuns
            bairro = re.sub(r'\b(perto|próximo|próx|de|da|do|dos|das)\b', '', bairro, flags=re.IGNORECASE)
            return bairro.strip()
    
    return None

udf_extrair_bairro = udf(extrair_bairro, StringType())

In [ ]:
# UDF para extrair complemento
def extrair_complemento(texto, numero, bairro):
    """Extrai complemento do endereço (apartamento, bloco, etc)"""
    if not texto:
        return None
    
    # Remove partes já identificadas
    texto_limpo = texto
    if numero:
        texto_limpo = re.sub(numero, '', texto_limpo, flags=re.IGNORECASE)
    if bairro:
        texto_limpo = re.sub(bairro, '', texto_limpo, flags=re.IGNORECASE)
    
    # Padrões de complemento
    padroes_complemento = [
        r'\b(apto|ap\.|apartamento|bloco|bl\.|casa|sala|loja|andar|andar\s+\d+)\b',
        r'\b(\d+[º°]?\s*andar)\b',
        r'\b(sala\s+\d+|loja\s+\d+)\b',
    ]
    
    for padrao in padroes_complemento:
        match = re.search(padrao, texto_limpo, re.IGNORECASE)
        if match:
            return match.group(0)
    
    return None

udf_extrair_complemento = udf(extrair_complemento, StringType())

In [ ]:
# Função principal para estruturar endereço livre
def estruturar_endereco_livre(df_enderecos_livres):
    """
    Estrutura endereços livres em componentes usando NER simplificado
    
    Args:
        df_enderecos_livres: DataFrame com endereços livres
    
    Returns:
        DataFrame estruturado com componentes do endereço
    """
    df_estruturado = df_enderecos_livres \
        .withColumn("numero", udf_extrair_numero(col("endereco_livre"))) \
        .withColumn("tipo_logradouro", udf_identificar_tipo(col("endereco_livre"))) \
        .withColumn("nome_logradouro", udf_extrair_nome_log(col("endereco_livre"), col("tipo_logradouro"))) \
        .withColumn("bairro", udf_extrair_bairro(col("endereco_livre"))) \
        .withColumn("complemento", udf_extrair_complemento(col("endereco_livre"), col("numero"), col("bairro"))) \
        .withColumn("endereco_estruturado", 
            concat_ws(" | ", 
                col("tipo_logradouro"),
                col("nome_logradouro"),
                col("numero"),
                col("bairro"),
                col("complemento")
            )
        ) \
        .withColumn("processado_em", current_timestamp()) \
        .select(
            col("id"),
            col("endereco_livre"),
            col("uf"),
            col("cidade"),
            col("tipo_logradouro"),
            col("nome_logradouro"),
            col("numero"),
            col("bairro"),
            col("complemento"),
            col("endereco_estruturado"),
            col("processado_em")
        )
    
    return df_estruturado

In [ ]:
# Exemplo de uso: Ler endereços livres e estruturar
# df_enderecos_livres = read_delta_table(PATH_ENDERECOS_LIVRES)

# Criar dados de exemplo para teste
dados_exemplo = [
    ("1", "entregar na r. do ouvidor, 120 perto da praça, centro, rj", "RJ", "Rio de Janeiro", None),
    ("2", "Av. Paulista, 1578, Bela Vista, São Paulo", "SP", "São Paulo", None),
    ("3", "Rua XV de Novembro número 50 apto 201 centro", "SP", "São Paulo", None),
]

df_exemplo = spark.createDataFrame(dados_exemplo, schema_endereco_livre)
df_exemplo.show(truncate=False)

In [ ]:
# Aplicar estruturação
df_estruturado = estruturar_endereco_livre(df_exemplo)
df_estruturado.show(truncate=False)

In [ ]:
# Salvar endereços estruturados na Camada Silver
# save_delta_table(df_estruturado, PATH_ENDERECOS_ESTRUTURADOS, mode="append", partition_by=["uf", "cidade"])

print("Processo de estruturação concluído!")